# Preprocessing Ablation Data
> Preprocessing amnioserosa ablation movies from Hara et al. 2016

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import torch
from os import path

import networkx as nx, pandas as pd
from simgnn.datautils import simple_moving_average,load_array, load_graph

import matplotlib.pyplot as plt
import matplotlib
plt.style.use('ggplot')
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (7,7) # use larger for presentation
matplotlib.rcParams['font.size']= 14 # use 14 for presentation

In [3]:
from torch_geometric.data import Data, DataLoader
from torch_geometric.utils import to_networkx, from_networkx
import torch_geometric.transforms as T

from simgnn.datasets import VertexDynamics, HaraMovies, CellData
from simgnn.transforms import Pos2Vec, ScaleVelocity, ScaleTension, ScalePressure

In [4]:
device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda')
dtype = torch.float32
print(f'Defaults:\n |-device: {device}\n |-dtype : {dtype}')

Defaults:
 |-device: cpu
 |-dtype : torch.float32


In [279]:
ablmov_basepath = path.abspath('../../../dataDIR/Hara_ablation_movies/')
data_base_path = './simgnn_data/hara_ablation/'

recoil_df = pd.read_csv(path.join(data_base_path,'hara_ablation_data.csv'))
recoil_df.head()

,Main folder,embryo,Edge ID,Recoil speed [um/sec],Edge Deformation [um/sec],Edge Behavior,Edge Rn,Rn Category,IsRosette,AblationFrame,# 1-neighbours,# 2-neighbours,Notes
0,140408_SqhGFP_UASDaCatRFP_c381Gal4,sample10,1,0.023877,0.020217,Expanding,1.104026,1.1-1.5,False,57,4,8,NaN
1,140411_Dacat_recoil,sample3,1,0.464255,-0.043337,Contracting,1.264302,1.1-1.5,False,47,4,9,NaN
2,140411_Dacat_recoil,sample5,1,0.452162,0.019286,Expanding,1.075047,1.0-1.1,False,50,4,8,NaN
3,140411_Dacat_recoil,sample6,1,0.283717,0.006938,Stable,1.331903,1.1-1.5,False,12,4,6,NaN
4,140415_Dacat_recoil,sample1,1,0.524569,-0.052628,Contracting,1.148226,1.1-1.5,False,30,4,8,NaN


### Read TIF files

In [5]:
import glob
from simgnn.hara import get_cell_colormap, read_tiff_stack

from simgnn.datautils import write_array, mknewdir
from simgnn.hara import label_bw_stack, extract_graph

In [28]:
base_path = path.abspath('../../../dataDIR/example_segmented_data/')

print('RFP channels (cell boundary):')
rfp = {path.basename(path.dirname(fname)) : read_tiff_stack(fname) 
       for fname in glob.glob(path.join(base_path,'*Seg*','*RFP*.tif'))}
print('Segmented boundaries:')
seg = {path.basename(path.dirname(fname)) : read_tiff_stack(fname) 
       for fname in glob.glob(path.join(base_path,'*Seg*','*Seg*.tif'))}

RFP channels (cell boundary):
Image stack shape: (61, 512, 511) -- trim boundaries: True
Image stack shape: (61, 512, 511) -- trim boundaries: True
Image stack shape: (61, 512, 511) -- trim boundaries: True
Segmented boundaries:
Image stack shape: (61, 512, 511) -- trim boundaries: True
Image stack shape: (61, 512, 511) -- trim boundaries: True
Image stack shape: (61, 512, 511) -- trim boundaries: True


### Extract Graphs from Amnioserosa Movies

In [29]:
root_dir = 'simgnn_data/hara_movies/raw/'
PROCESS_HARA_MOVIE = False # set to true to process Hara AS tissue movies

if PROCESS_HARA_MOVIE:
    for bw_img in seg:
        edges_index, node2cell_index, v_pos = extract_graph(seg[bw_img], label_bw_stack(seg[bw_img])[0])
        edge_lengths = np.sqrt(((v_pos[:,edges_index[1],:] - v_pos[:,edges_index[0],:])**2).sum(axis=-1,keepdims=True))

        dataDir = path.join(root_dir, bw_img)
        mknewdir(dataDir);

        write_array(path.join(dataDir, 'edges_index.npy'), edges_index)
        write_array(path.join(dataDir, 'node2cell_index.npy'), node2cell_index)
        write_array(path.join(dataDir, 'vtx_pos.npy'), v_pos)
        write_array(path.join(dataDir, 'edge_Length.npy'), edge_lengths)